# Levenberg-Marquardt mothod for ordinary differential equations

The purpose is to produce
1. $\varphi(\mathbf{z})$ that produces $n\times(p+1)$-dimension vector with input arguments of function, initial condition $\mathbf{y}_0$, and parameters $\mathbf{k}$.
2. Levenberg-Marquardt parameter estimation procedure.

In [4]:
import numpy as np
from numpy import linalg as LA
from scipy.integrate import solve_ivp, odeint

## ODE model and objective function
For the intial value problem
\begin{equation}
\frac{\text{d}\mathbf{y}(t)}{\text{d}t}=\mathbf{f}(\mathbf{y}(t),\mathbf{k});\hspace{10mm}\mathbf{y}(t_0)=\mathbf{y}_0
\end{equation}
where $\mathbf{k}$ is a parameter vector with $p$ elements, the objective function with the measurements $\hat{\mathbf{y}}_i$ ($i=1,\cdots,N$) and the weighting matrix $\mathbf{Q}_i$ is
\begin{equation}
S(\mathbf{k})=\frac{1}{2}\sum_{i=1}^N\left[\hat{\mathbf{y}}_i-\mathbf{y}(t_i,\mathbf{k})\right]^\top\mathbf{Q}_i\left[\hat{\mathbf{y}}_i-\mathbf{y}(t_i,\mathbf{k})\right]=\frac{1}{2}\sum_{i=1}^N\mathbf{r}_i(\mathbf{k})^\top\mathbf{Q}_i\mathbf{r}_i(\mathbf{k})
\end{equation}
where $S(\mathbf{k}):\mathbb{R}^p\to\mathbb{R}$ and $\mathbf{r}_i(\mathbf{k})=\hat{\mathbf{y}}_i-\mathbf{y}(t_i,\mathbf{k})$ is the residuals. The gradient of the objective function is
\begin{equation}
\mathbf{g}=\frac{\partial S(\mathbf{k})}{\partial\mathbf{k}}=-\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{r}_i
\end{equation}
and the Hessian is
\begin{align}
\mathbf{H}&=\frac{\partial^2S(\mathbf{k})}{\partial\mathbf{k}^2}=\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i-\sum_{i=1}^N\frac{\partial\mathbf{J^\top}_i}{\partial\mathbf{k}}\mathbf{Q}_i\mathbf{r}_i\\
&\simeq\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i
\end{align}
where $\mathbf{J}_i(\mathbf{k})=\frac{\partial\mathbf{y}(t_i,\mathbf{k})}{\partial\mathbf{k}}$ is the Jacobian. The approximation neglecting final term is valid since it is often the case that the residuals $\mathbf{r}_i$ are small.

## Calculation of Jacobian in ODE models
In ODE models, the sensitivity matrix cannot be obtained by a simple differentiation. Instead, we can get differential equation for $\mathbf{J}$.
Differentiate both side of $ \frac{\text{d}\mathbf{y}}{\text{d}t}=\mathbf{f}(\mathbf{y}(t),\mathbf{k})$ and apply the chain rule,
\begin{align}
\frac{\partial}{\partial\mathbf{k}}\left(\frac{\text{d}\mathbf{y}}{\text{d}t}\right)=\frac{\text{d}}{\text{d}t}\left(\frac{\partial\mathbf{y}}{\partial\mathbf{k}}\right)&=\frac{\partial}{\partial\mathbf{k}}\mathbf{f}(\mathbf{y}(t),\mathbf{k})\\
&=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\frac{\partial\mathbf{y}}{\partial\mathbf{k}}+\frac{\partial\mathbf{f}}{\partial\mathbf{k}}\frac{\partial\mathbf{k}}{\partial\mathbf{k}}\\
&=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\frac{\partial\mathbf{y}}{\partial\mathbf{k}}+\frac{\partial\mathbf{f}}{\partial\mathbf{k}}
\end{align}
Hence,
\begin{equation*}
\frac{\text{d}\mathbf{J}(t)}{\text{d}t}=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{J}(t)+\frac{\partial\mathbf{f}}{\partial\mathbf{k}};\hspace{10mm}\mathbf{J}(t_0)=0
\end{equation*}


## Contruction of ODE system with Jacobian
The Jacobian or the sensitivity matrix is
\begin{equation}
\mathbf{J}(t)=\frac{\partial\mathbf{y}}{\partial\mathbf{k}}=\left[\frac{\partial\mathbf{y}}{\partial k_1},\cdots,\frac{\partial\mathbf{y}}{\partial k_p}\right]=[\mathbf{g}_1,\cdots,\mathbf{g}_p]
\end{equation}
where $\mathbf{g}_j$ represents $n$-dimensional vector which is the sensitivity coefficients of the state variables with respect to parameter $k_j$. Each of $\mathbf{g}_j$ satisfies the differential equation for Jacobian such that
\begin{equation*}
\frac{\text{d}\mathbf{g}_j(t)}{\text{d}t}=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{g}_j+\frac{\partial\mathbf{f}}{\partial k_j};\hspace{10mm}\mathbf{g}_j(t_0)=0;\hspace{10mm}j=1,\cdots,p
\end{equation*}
We generate $n\times(p+1)$-dimensional differential equations system
\begin{equation*}
\frac{d\mathbf{z}}{dt}=\varphi(\mathbf{z})
\end{equation*}
$\mathbf{z}$ is $n\times(p+1)$-dimensional vector
\begin{equation*}
\mathbf{z}=\begin{bmatrix} \mathbf{x}(t)\\
                          \frac{\partial\mathbf{y}}{\partial k_1}\\
                          \vdots\\
                          \frac{\partial\mathbf{y}}{\partial k_p}
\end{bmatrix}
=\begin{bmatrix} \mathbf{y}(t)\\
                 \mathbf{g}_1(t)\\
                 \vdots\\
                 \mathbf{g}_p(t)
\end{bmatrix}
\end{equation*}
$\mathbf{\varphi}(\mathbf{z})$ is $n\times(p+1)$-dimensional vector function

\begin{equation*}
\mathbf{\varphi}(\mathbf{z})=\begin{bmatrix}
\mathbf{f}(\mathbf{y},\mathbf{k})\\
\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{g}_1(t)+\frac{\partial\mathbf{f}}{\partial k_1}\\
\vdots\\
\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{g}_p(t)+\frac{\partial\mathbf{f}}{\partial k_p}
\end{bmatrix}
\end{equation*}

To get the Jacobian for all $t_i$, $\varphi(\mathbf{z}_i)$ should be solved for $t_i,~~i=1,2,\cdots,N$.


### Arrange of the solution
The Jacobian $\textbf{J}$ is obtained by integration of $\varphi(z)$. Integration of $\varphi(z)$ returns $n\times(p+1)$ vector
\begin{equation*}
\textbf{z}=\begin{bmatrix}
\textbf{y}\\
\textbf{g}_1\\
\textbf{g}_2\\
\vdots\\
\textbf{g}_p
\end{bmatrix}
\end{equation*}
where
\begin{equation*}
\textbf{g}_j=\begin{bmatrix}
\frac{\partial y_1}{\partial k_j}\\
\frac{\partial y_2}{\partial k_j}\\
\vdots\\
\frac{\partial y_n}{\partial k_j}
\end{bmatrix},\hspace{20mm}j=1,\cdots,p
\end{equation*}
To compute the Hessian
\begin{equation*}
\mathbf{H}=\sum_{i=1}^N\mathbf{J}(t_i)^\top\mathbf{Q}_i\mathbf{J}(t_i)
\end{equation*}
the Jacobian for all measurement time $\mathbf{J}(t_1),\cdots,\mathbf{J}(t_N)$, should be returned.
The ODE solver for initial value problem returns $[n\times(p+1)]\times N$ matrix
\begin{equation*}
Z=\begin{bmatrix}
\mathbf{y}(t_1)&\mathbf{y}(t_2)&\cdots&\mathbf{y}(t_N)\\
\mathbf{g}_1(t_1)&\mathbf{g}_1(t_2)&\cdots&\mathbf{g}_1(t_N)\\
\vdots&&\ddots&\vdots\\
\mathbf{g}_p(t_1)&\mathbf{g}_p(t_2)&\cdots&\mathbf{g}_p(t_N)
\end{bmatrix}
\end{equation*}
This matrix would be refomulated for
\begin{equation*}
\textbf{Y}=\begin{bmatrix}
\mathbf{y}(t_1)&\mathbf{y}(t_2)&\cdots&\mathbf{y}(t_N)
\end{bmatrix}
\end{equation*}
and
\begin{align*}
\mathcal{J}=&\begin{bmatrix}
\mathbf{J}(t_1),\mathbf{J}(t_2),\cdots,\mathbf{J}(t_N)
\end{bmatrix}\\
=&\left[\begin{bmatrix}
\textbf{g}_1(t_1)&\textbf{g}_2(t_1)&\cdots&\textbf{g}_p(t_1)
\end{bmatrix}
,\cdots,\begin{bmatrix}
\textbf{g}_1(t_N)&\textbf{g}_2(t_N)&\cdots&\textbf{g}_p(t_N)
\end{bmatrix}\right].
\end{align*}
Note that $\mathbf{Y}$ is an $n\times N$ matrix and $\mathcal{J}$ is an $n\times p\times N$ tensor.

### Integration caveats
For ODE models, it is unnecessary to integrate the ODEs for the entire time domain $[t_1,t_N]$. Once the objective function $S(\mathbf{k+h})$ becomes greater than $S(\mathbf{k})$, the integration should be stopped and retried with another $\mathbf{h}$ or with appropriate line search. This can prevent not only the unnecessary computation but also numerical instability in integration ODEs with the parameters far from the solution which can induce the computer overflow.

In [11]:
def integration(ode,z0,k,t,n,p):
    def dzdt(t,z):
        return phi_ode(ode,z,k,n,p)
    solution = solve_ivp(dzdt,[t[0],t[-1]],z0,method='Radau',t_eval=t)
    return solution.y, solution.success
    

def integ_onestep(ode,z0,k,t,n,p):
    def dzdt(t,z):
        return phi_ode(ode,z,k,n,p)
    solution = solve_ivp(dzdt,[t[0],t[-1]],z0,method='Radau',t_eval=t)
    z = solution.y[:,-1]
    return z, solution.success

def phi_ode(ode,z,k,n,p):
    y = z[0:n]
    J = z[n:].reshape((p,n)).transpose()
    phiz = np.empty(n*(p+1))
    dfdy = dfdy_ode(ode,y,k,n)
    dfdk = dfdk_ode(ode,y,k,n,p)
    dJdt = dfdy@J+dfdk
    phiz[0:n] = ode(y,k)
    phiz[n:] = dJdt.transpose().flatten()
    return phiz

def dfdy_ode(ode,y,k,n):
    h = 1e-8
    y = y.astype(np.float)
    if np.isscalar(y):
        dfdy = (ode(y+h,k)-ode(y-h,k))/(2*h)
        return dfdy
    else:
        dfdy = np.empty((n,n))
        for i in range(n):
            yr = y.copy()
            yl = y.copy()
            yr[i] += h
            yl[i] -= h
            dfdy[i] = (ode(yr,k)-ode(yl,k))/(2*h)
        return dfdy.transpose()
    return

def dfdk_ode(ode,y,k,n,p):
    h = 1e-8
    if p == 1:
        dfdk = (ode(y,k+h)-ode(y,k-h))/(2*h)
        return dfdk.reshape(n,1)
    else:
        k = k.astype(np.float)
        dfdk = np.empty((p,n))
        for i in range(p):
            kr = k.copy()
            kl = k.copy()
            kr[i] += h
            kl[i] -= h
            dfdk[i] = (ode(y,kr)-ode(y,kl))/(2*h)
        return dfdk.transpose()
    return

## Talyor expansion of scalar function of several variables
The line passes $\mathbf{x}_0$ along the direction $\mathbf{s}$ is the set of points satisfying $\mathbf{x}(\alpha)=\mathbf{x}_0+\alpha\mathbf{s}$ for all $\alpha$. For a scalar value function $f(\mathbf{x})$, the directional derivative of $f$ at $\mathbf{x}_0$ in the direction of $\mathbf{s}$ is,
\begin{equation}
D_\mathbf{s}f(\mathbf{x}_0)=\lim_{\alpha\to0}\frac{f(\mathbf{x}_0+\alpha\mathbf{s})-f(\mathbf{x}_0)}{\alpha}=\frac{\text{d}f(\textbf{x})}{\text{d}\alpha}\bigg|_{\textbf{x}=\textbf{x}_0}
\end{equation}
By the chain rule,
\begin{equation}
\frac{\text{d}}{\text{d}\alpha}=\sum_i\frac{\partial}{\partial x_i}\frac{\text{d}x_i(\alpha)}{\text{d}\alpha}=\sum_is_i\frac{\partial}{\partial x_i}=\mathbf{s^\top \nabla}
\end{equation}
so the slope of $f$ along any line $\mathbf{x}(\alpha)$ is
\begin{equation}
\frac{\text{d}f}{\text{d}\alpha}=\mathbf{s}^\top\nabla f=\nabla f^\top\mathbf{s}
\end{equation}
and the curvature along the line is
\begin{equation}
\frac{\text{d}^2 f}{\text{d}\alpha^2}=\frac{\text{d}}{\text{d}\alpha}\frac{\text{d}f}{\text{d}\alpha}=\mathbf{s}^\top\nabla(\nabla f^\top\mathbf{s})=\mathbf{s}^\top\nabla^2f\mathbf{s}.
\end{equation}.
The Taylor expansion of multiple variables is
\begin{equation}
f(\mathbf{x}_0+\alpha\mathbf{s})=f(\mathbf{x}_0)+\alpha\mathbf{s}^\top\nabla f(\mathbf{x}_0)+\frac{1}{2}\alpha^2\mathbf{s}^\top\left[\nabla^2f(\mathbf{x}_0)\right]\mathbf{s}+\cdots
\end{equation}
or
\begin{equation}
f(\mathbf{x}_0+\mathbf{h})=f(\mathbf{x}_0)+\mathbf{h}^\top\nabla f(\mathbf{x}_0)+\frac{1}{2}\mathbf{h}^\top\left[\nabla^2f(\mathbf{x}_0)\right]\mathbf{h}+\cdots.
\end{equation}

## Gauss-Newton method
The Taylor expansion of $\mathbf{r}_i$ respect to $\mathbf{h}$ is
$$\mathbf{r}_i(\mathbf{k+h})=\mathbf{r}_i(\mathbf{k})+\frac{\partial\mathbf{r}_i(\mathbf{k})}{\partial\mathbf{k}}\mathbf{h}+\mathcal{O}(\mathbf{h^\top h})=\mathbf{r}_i(\mathbf{k})-\mathbf{J}_i(\mathbf{k})\mathbf{h}+\mathcal{O}(\mathbf{h^\top h}).$$  
In Gauss-Newton (GN) method, $\mathbf{r}_i$ is approximated to a linear model
\begin{equation}
\mathbf{l}_i=\mathbf{r}_i(\mathbf{k})-\mathbf{J}_i(\mathbf{k})\mathbf{h}
\end{equation}
Inserting $\mathbf{l}_i$ to the objective function yields
\begin{align}
S(\mathbf{k+h})&\simeq L(\mathbf{h})=\frac{1}{2}\sum_{i=1}^N\mathbf{l}_i^\top(\mathbf{h})\mathbf{Q}_i\mathbf{l}_i(\mathbf{h})\\
    &=\frac{1}{2}\sum_{i=1}^N\left[\mathbf{r}_i\mathbf{(k)}^\top\mathbf{Q}_i\mathbf{r}_i\mathbf{(k)}-\mathbf{r}_i^\top\mathbf{Q}_i\mathbf{J}_i\mathbf{(k)h}-\mathbf{h}^\top\mathbf{J}_i\mathbf{(k)}^\top\mathbf{Q}_i\mathbf{r}_i(\mathbf{k})+\mathbf{h}^\top\mathbf{J}_i\mathbf{(k)}^\top\mathbf{Q}_i\mathbf{J}_i\mathbf{(k)}\mathbf{h}\right]\\
    &=S(\mathbf{k})-\sum_{i=1}^N\mathbf{h^\top J}_i^\top\mathbf{Q}_i \mathbf{r}_i+\frac{1}{2}\sum_{i=1}^N\mathbf{h^\top J}_i^\top\mathbf{Q}_i\mathbf{J}_i\mathbf{h}
\end{align}
since $\mathbf{r}_i^\top\mathbf{Q}_i\mathbf{J}_i\mathbf{(k)h}=\mathbf{h}^\top\mathbf{J}_i\mathbf{(k)}^\top\mathbf{Q}_i\mathbf{r}_i(\mathbf{k})$.  
The gradient and Hessian of $L$ are
\begin{equation}
\mathbf{L'(h)}=\sum_{i=1}^N\left[-\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{r}_i+\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i\mathbf{h}\right]\hspace{20mm}\mathbf{L''(h)}=\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i
\end{equation}
The Hessian is independent of $\mathbf{h}$, symmetric and positive definite if $\mathbf{J}$ has full rank. Hence $L$ is minimum when $\mathbf{L'(h)}$ is zero vector. The step $\mathbf{h}$ can be calculated by,
\begin{equation}
\left[\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i\right]\mathbf{h}=\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{r}_i=\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\left(\hat{\mathbf{y}}_i-\mathbf{y}(t_i,\mathbf{k})\right)
\end{equation}
so that
\begin{equation}
\mathbf{Hh=-g}
\end{equation}
GN method is not quadratic convergent if $\mathbf{r}_i$ is not zero around the solution.


## Stopping criteria
When the algorithm can be expected to converge rapidly, the criteria
\begin{equation}
S^{(k)}-S^{(k+1)}\leq\varepsilon
\end{equation}
works well. For algorithms converge less rapidly, a test based on
\begin{equation}
\Vert\mathbf{g}^{(k)}\Vert\leq\varepsilon
\end{equation}
is more appropriate.

## Levenberg-Marquardt method
In Levenberg-Marquardt (LM) method, GN method is used with a damping term,
\begin{equation*}
\left[\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i+\mu\mathbf{I}\right]\mathbf{h}_\text{lm}=\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{r}_i
\end{equation*}
For large $\mu$, $\mathbf{h}_\text{lm}\simeq-\frac{1}{\mu}\mathbf{L'(0)}$ is a short step in the steepest descent direction. This is good if the estimation is far from the solution. If $\mu$ is very small, LM method is nearly GN method which is almost quadratic convergent if $S(\mathbf{k})$ is close to zero.
### Initial $\mu$
The initial value of $\mu$ is maximum diagonal element of $\mathbf{H}_0=\sum\mathbf{J}_i(\mathbf{k}_0)^\top\mathbf{Q}_i\mathbf{J}_i(\mathbf{k}_0)$,
\begin{equation*}
\mu_0=\tau\cdot\max[\text{diagonal}(\mathbf{H}_0)]
\end{equation*}
where $\tau$ is small such as $10^{-6}$ if $\mathbf{k}_0$ is believed to be a good approximation or large such as $10^{-3}$ or $1$ otherwise.
### Gain ratio
The updating of $\mu$ is controlled by the gain ratio
\begin{equation*}
\rho=\frac{S(\mathbf{k})-S(\mathbf{k+h}_\text{lm})}{L(\mathbf{0})-L(\mathbf{h}_\text{lm})}
\end{equation*}
The denominator is the gain predicted by the linear model,
\begin{align*}
L(\mathbf{0})-L(\mathbf{h}_\text{lm})=&\mathbf{h}_\text{lm}^\top\sum_{i=1}^N\mathbf{J}_i\mathbf{Q}_i\mathbf{r}_i-\frac{1}{2}\mathbf{h}^\top_\text{lm}\left[\sum_{i=1}^N\mathbf{J}_i\top\mathbf{Q}_i\mathbf{J}_i\right]\mathbf{h}_\text{lm}\\
                   =&\frac{1}{2}\mathbf{h}_\text{lm}^\top\left(2\sum_{i=1}^N\mathbf{J}_i\mathbf{Q}_i\mathbf{r}_i-\left[\sum_{i=1}^N\mathbf{J}^\top_i\mathbf{Q}_i\mathbf{J}_i+\mu\mathbf{I}\right]\mathbf{h}_\text{lm}+\mu\mathbf{I}\mathbf{h}_\text{lm}\right)\\
                   =&\frac{1}{2}\mathbf{h}_\text{lm}^\top\left(\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{r}_i+\mu\mathbf{h}_\text{lm}\right)\\
                   =&\frac{1}{2}\mathbf{h}_\text{lm}^\top\left(-\mathbf{g}+\mu\mathbf{h}_\text{lm}\right)
\end{align*}
A large value of $\rho$ indicates that $L(\mathbf{h}_\text{lm})$ is a good approximation of $S(\mathbf{k+h}_\text{lm})$ so $\mu$ can be decreased so that LM is closer to GN method.



### Reduced Jacobian
The Hessian $\mathbf{H}$ might be ill-conditioned if the paramters differ by several order of magnitude. To overcome numerical problem, reduced Jacobian $\mathbf{J}_\text{R}$ is defined as
$$\mathbf{J}_\text{R}=\mathbf{JK}$$
where $\mathbf{K}=\text{diag}(k_1,\cdots,k_p)$. Then we can solve
$$\mathbf{H}_\text{R}\mathbf{h}_\text{R}=-\mathbf{g}_\text{R}$$
where $\mathbf{H}_\text{R}=\mathbf{KHK}$ and $\mathbf{g}_\text{R}=\mathbf{Kg}$.
We can restore $\textbf{h}$ from $\textbf{h=Kh}_\text{R}$

## Algorithm of LM method




In [5]:
def gnlm(ode,yhat,Q,k0,t,opts=[1e-3,1e-8,1e-8,1000]):
    # Input arguments

    # opts = [tau, tolg, tolk, max_iter]
    #
    # Outputs
    # output = [k,Y,info]
    # k : parameters
    # Y : results with k
    # info = [it,ter]
    # it : Number of iterations
    # ter : Termination criteria 1: gradient 2: change in h 3: maximum iteration

    try:
        stop = False
        nu = 2
        it = 0 
        rho = 0
        ter = 'm'
        N = np.size(t)
        if np.ndim(yhat) == 1:
            scalar = True
            n = 1
            assert N == np.size(yhat), "Dimension mismatch with yhat and t"
        else:
            scalar = False
            n = np.size(yhat,0)
            assert N == np.size(yhat,1), "Dimension mismatch with yhat and t"
        p = np.size(k0)
        k = k0.copy()
        Y,Jt,S,r,fail = checkSrJ(ode,yhat,k,t,n,p,N,Q,1e8,scalar)
        assert not fail, "Huge residuals"
        S0 = S
        r0 = r.copy()
        H,g = Hg(Jt,Q,r,p,N)
        gn = LA.norm(g,np.inf)
        stop = bool(gn < opts[1])
        if stop:
            ter = 'g'
            print("First guess was correct!")
        mu = opts[0]*max(np.diag(H))
        print('Iter | Obj func | step size | gradient |   mu   |   rho')
        print("{0:5d}|{1:10.4e}|   Not cal |  Not cal |{2:8.1e}| Not cal"
              .format(it,S,mu))
        while (not stop) and (it<=opts[3]):
            fail = False
            it += 1
            h,mu = cholsolve(H,-g,mu,p)
            hn = LA.norm(h,2)
            kn = LA.norm(k,2)
            if hn <= opts[2]*(kn+opts[2]):
                stop = True
                ter = 'h'
            else:
                k_new = k + h
                Y,Jt,S,r,fail = checkSrJ(ode,yhat,k_new,t,n,p,N,Q,S0,scalar)
                dL = h@(mu*h-g)/2
                if dL>0 and not fail:
                    df = dF(r0,r,Q,n,N)
                    rho = df/dL
                    k = k_new 
                    S0 = S
                    r0 = r.copy()
                    H, g = Hg(Jt,Q,r,p,N)
                    gn = LA.norm(g,np.inf) 
                    if gn < opts[1]:
                        stop = True
                        ter = 'g'
                    mu *= max(1/3,1-(2*rho-1)**3)
                    nu = 2
                else:
                    mu *= nu
                    nu *= 2
            if rho == 0:
                print("{0:5d}|{1:10.4e}|{2:11.3e}|{3:10.2e}|{4:8.1e}| Not cal"
                        .format(it,S,hn,gn,mu))
            else:
                print("{0:5d}|{1:10.4e}|{2:11.3e}|{3:10.2e}|{4:8.1e}|{5:8.1e}"
                        .format(it,S,hn,gn,mu,rho))
        info = [it,ter]
        output = [k,Y,info]
        return output
    except OverflowError as oerr:
        print(oerr.args)
        return
    except AssertionError as aerr:
        print(aerr.args)
        return   

    
def checkSrJ(ode,yhat,k,t,n,p,N,Q,S0,scalar):
    # initial condition J0 = 0
    if scalar:
        y0 = yhat[0]
    else:
        y0 = yhat[:,0]
    fail = False    
    r = np.zeros((n,N))    
    Z = np.zeros((n*(p+1),N))
    Z[0:n,0] = y0.copy()
    S = 0
    i = 0
    Z, suc = integration(ode,Z[0],k,[t[0],t[-1]],n,p)
    if not suc:
        S = S0
        fail = True
    Y = Z[0:n]
    S = 0
    r = yhat-Y
    if np.size(Q) == 1:
        S = np.sum(r**2)
    else:
        for i in range(N):
            S += r[:,i]@Q@r[:,i]
    if S > S0:
        S = S0
        fail = True
    J = Z[n:]
    Jt = np.hsplit(J,N)
    for i in range(N):
        Jt[i] = Jt[i].reshape(p,n).transpose()
    return Y,Jt,S,r,fail

def checkSrJ_step(ode,yhat,k,t,n,p,N,Q,S0,scalar):
    # initial condition J0 = 0
    if scalar:
        y0 = yhat[0]
    else:
        y0 = yhat[:,0]
        
    r = np.zeros((n,N))    
    Z = np.zeros((n*(p+1),N))
    Z[0:n,0] = y0.copy()
    S = 0
    i = 0
    fail = False
    
    while (not fail) and i<N-1:
        Z[:,i+1],suc = integ_onestep(ode,Z[:,i],k,[t[i],t[i+1]],n,p)
        if not suc:
            S = S0
            fail = True
        else:
            if scalar:            
                r[:,i+1] = yhat[i+1]-Z[0,i+1]
            else:
                r[:,i+1] = yhat[:,i+1]-Z[0:n,i+1] 
            S += r[:,i+1]@Q@r[:,i+1]/2
            if S>S0:
                S = S0
                fail = True
        i += 1
        
    Y = Z[0:n]
    J = Z[n:]
    Jt = np.hsplit(J,N)
    for i in range(N):
        Jt[i] = Jt[i].reshape(p,n).transpose()
    return Y,Jt,S,r,fail    

def Hg(Jt,Q,r,p,N):
    H = np.zeros((p,p))
    g = np.zeros(p)
    for i in range(N):
        JQ = Jt[i].T@Q
        H += JQ@Jt[i]
        g -= JQ@r[:,i]
    return H,g


def dF(r0,r,Q,n,N):
    dS = 0
    if n == 1:
        dS = (r0[0]-r[0])@(r0[0]+r[0])/2
    else:
        for i in range(N):
            dS += (r0[:,i]-r[:,i])@Q@(r0[:,i]+r[:,i])/2
    return dS
    
def cholesky(A,p):
    C = np.zeros((p,p))
    j = 0
    pd = True
    while pd and j<p:
        sum = 0
        for k in range(j):
            sum += C[j][k]**2
        d = A[j][j]-sum
        if d>0:
            C[j][j] = np.sqrt(d)
            for i in range(j,p):
                sum = 0
                for k in range(j):
                    sum += C[i][k]*C[j][k]
                C[i][j] = (A[i][j]-sum)/C[j][j]
        else:
            pd = False
        j += 1
    return C,pd

def cholsolve(A,b,mu,p):
    I = np.eye(p)
    mA = np.amax(abs(A))
    pd = False
    while pd == False:
        C,pd = cholesky(A+mu*I,p)
        # check for near singularity
        if pd == True:
            pd = (1/LA.cond(C,1)>=1e-15)
        if pd == False:
            mu = max(10*mu,np.finfo(float).eps*mA)
    # CC^Tx = b
    z = np.zeros(p)
    x = np.zeros(p)
    # Forward C^Tx = z
    for i in range(p):
        sum = 0
        for j in range(i):
            sum += C[i][j]*z[j]
        z[i] = (b[i]-sum)/C[i][i]
    # Backward Cz = b
    for i in reversed(range(p)):
        sum = 0
        for j in range(i,p):
            sum += C[j][i]*x[j]
        x[i] = (z[i]-sum)/C[i][i]
    return x,mu

## Misc
The stopping criteria $\Vert\mathbf{g}\Vert\leq\varepsilon$ has disadvantage that it is not easy for the user to know what magnitude to choose for $\varepsilon$ and does not have certain important invariance properties.

For the affine transformation
\begin{equation}
\mathbf{y=Ax+a}
\end{equation}
the gradient relation is
\begin{equation}
\mathbf{g}_x=\mathbf{A^\top g}_y
\end{equation}
and the Hessian relation is
\begin{equation}
\mathbf{G}_x=\mathbf{A^\top G}_y\mathbf{A}
\end{equation}
Newton-like method with the soft line search is invariant. However the steepest descent method is not invariant since $\mathbf{I}$ does not transform correctly and so is modified Newton method since $\mathbf{H}+\mu\mathbf{I}$ does not transform correctly.